# 1. RNN을 이용하여 텍스트 생성하기
> **1) 데이터에 대한 이해와 전처리**

In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras.utils import to_categorical

text = """경마장에 있는 말이 뛰고 있다\n
그의 말이 법이다\n
가는 말이 고와야 오는 말이 곱다\n"""

In [2]:
# 단어 집합 생성
t = Tokenizer()
t.fit_on_texts([text])

vocab_size = len(t.word_index) + 1
print('단어 집합의 크기: %d' % vocab_size)

print(t.word_index)

단어 집합의 크기: 12
{'말이': 1, '경마장에': 2, '있는': 3, '뛰고': 4, '있다': 5, '그의': 6, '법이다': 7, '가는': 8, '고와야': 9, '오는': 10, '곱다': 11}


In [3]:
# 레이블을 분리하지 않은 훈련 데이터
sequences = list()
for line in text.split('\n'):
    encoded = t.texts_to_sequences([line])[0]
    
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)
        
print('학습에 사용할 샘플의 개수: %d' % len(sequences))

print(sequences)

학습에 사용할 샘플의 개수: 11
[[2, 3], [2, 3, 1], [2, 3, 1, 4], [2, 3, 1, 4, 5], [6, 1], [6, 1, 7], [8, 1], [8, 1, 9], [8, 1, 9, 10], [8, 1, 9, 10, 1], [8, 1, 9, 10, 1, 11]]


In [4]:
# padding
max_len = max(len(l) for l in sequences)
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')

print(sequences)

[[ 0  0  0  0  2  3]
 [ 0  0  0  2  3  1]
 [ 0  0  2  3  1  4]
 [ 0  2  3  1  4  5]
 [ 0  0  0  0  6  1]
 [ 0  0  0  6  1  7]
 [ 0  0  0  0  8  1]
 [ 0  0  0  8  1  9]
 [ 0  0  8  1  9 10]
 [ 0  8  1  9 10  1]
 [ 8  1  9 10  1 11]]


In [5]:
# 샘플 마지막 단어를 레이블로 분리
sequences = np.array(sequences)
X = sequences[:, :-1]
y = sequences[:, -1]

print(X)
print(y)

[[ 0  0  0  0  2]
 [ 0  0  0  2  3]
 [ 0  0  2  3  1]
 [ 0  2  3  1  4]
 [ 0  0  0  0  6]
 [ 0  0  0  6  1]
 [ 0  0  0  0  8]
 [ 0  0  0  8  1]
 [ 0  0  8  1  9]
 [ 0  8  1  9 10]
 [ 8  1  9 10  1]]
[ 3  1  4  5  1  7  1  9 10  1 11]


In [6]:
# 레이블 원-핫 인코딩
y = to_categorical(y, num_classes=vocab_size)

print(y)

[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


> **2) 모델 설계하기**

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN

In [8]:
model = Sequential()
# 단어의 임베딩 벡터는 10차원, 은닉 상태 크기는 32인 Vanilla RNN
model.add(Embedding(vocab_size, 10, input_length=max_len-1))
model.add(SimpleRNN(32))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=200, verbose=2)

Epoch 1/200
1/1 - 1s - loss: 2.4705 - accuracy: 0.1818
Epoch 2/200
1/1 - 0s - loss: 2.4570 - accuracy: 0.1818
Epoch 3/200
1/1 - 0s - loss: 2.4434 - accuracy: 0.2727
Epoch 4/200
1/1 - 0s - loss: 2.4297 - accuracy: 0.2727
Epoch 5/200
1/1 - 0s - loss: 2.4159 - accuracy: 0.4545
Epoch 6/200
1/1 - 0s - loss: 2.4018 - accuracy: 0.4545
Epoch 7/200
1/1 - 0s - loss: 2.3873 - accuracy: 0.4545
Epoch 8/200
1/1 - 0s - loss: 2.3724 - accuracy: 0.4545
Epoch 9/200
1/1 - 0s - loss: 2.3569 - accuracy: 0.5455
Epoch 10/200
1/1 - 0s - loss: 2.3408 - accuracy: 0.5455
Epoch 11/200
1/1 - 0s - loss: 2.3240 - accuracy: 0.4545
Epoch 12/200
1/1 - 0s - loss: 2.3065 - accuracy: 0.4545
Epoch 13/200
1/1 - 0s - loss: 2.2882 - accuracy: 0.3636
Epoch 14/200
1/1 - 0s - loss: 2.2690 - accuracy: 0.3636
Epoch 15/200
1/1 - 0s - loss: 2.2490 - accuracy: 0.3636
Epoch 16/200
1/1 - 0s - loss: 2.2281 - accuracy: 0.3636
Epoch 17/200
1/1 - 0s - loss: 2.2064 - accuracy: 0.3636
Epoch 18/200
1/1 - 0s - loss: 2.1839 - accuracy: 0.3636
E

Epoch 147/200
1/1 - 0s - loss: 0.3789 - accuracy: 0.9091
Epoch 148/200
1/1 - 0s - loss: 0.3734 - accuracy: 0.9091
Epoch 149/200
1/1 - 0s - loss: 0.3680 - accuracy: 0.9091
Epoch 150/200
1/1 - 0s - loss: 0.3627 - accuracy: 0.9091
Epoch 151/200
1/1 - 0s - loss: 0.3575 - accuracy: 0.9091
Epoch 152/200
1/1 - 0s - loss: 0.3523 - accuracy: 0.9091
Epoch 153/200
1/1 - 0s - loss: 0.3472 - accuracy: 0.9091
Epoch 154/200
1/1 - 0s - loss: 0.3422 - accuracy: 0.9091
Epoch 155/200
1/1 - 0s - loss: 0.3372 - accuracy: 0.9091
Epoch 156/200
1/1 - 0s - loss: 0.3323 - accuracy: 0.9091
Epoch 157/200
1/1 - 0s - loss: 0.3275 - accuracy: 0.9091
Epoch 158/200
1/1 - 0s - loss: 0.3227 - accuracy: 0.9091
Epoch 159/200
1/1 - 0s - loss: 0.3179 - accuracy: 0.9091
Epoch 160/200
1/1 - 0s - loss: 0.3133 - accuracy: 0.9091
Epoch 161/200
1/1 - 0s - loss: 0.3087 - accuracy: 0.9091
Epoch 162/200
1/1 - 0s - loss: 0.3041 - accuracy: 0.9091
Epoch 163/200
1/1 - 0s - loss: 0.2996 - accuracy: 0.9091
Epoch 164/200
1/1 - 0s - loss: 

In [9]:
def sentence_generation(model, t, current_word, n):
    init_word = current_word
    sentence = ''
    
    for _ in range(n):
        encoded = t.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen=5, padding='pre')
        result = model.predict_classes(encoded, verbose=0)
        
        for word, index in t.word_index.items():
            if index == result:
                break
        current_word = current_word + ' ' + word
        sentence = sentence + ' ' + word
        
    sentence = init_word + sentence
    return sentence

In [10]:
print(sentence_generation(model, t, '경마장에', 4))

C:\Users\USER\anaconda3\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


경마장에 있는 말이 뛰고 있다


In [11]:
print(sentence_generation(model, t, '그의', 2))

그의 말이 법이다


In [12]:
print(sentence_generation(model, t, '가는', 5))

가는 말이 고와야 오는 말이 곱다


# 2. LSTM을 이용하여 텍스트 생성하기
> **1) 데이터에 대한 이해와 전처리**

In [13]:
import numpy as np
import pandas as pd
from string import punctuation
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [14]:
df = pd.read_csv('./ArticlesApril2018.csv')
df.head()

,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,5adf6684068401528a2aa69b,781,By JOHN BRANCH,article,Former N.F.L. Cheerleaders’ Settlement Offer: ...,"['Workplace Hazards and Violations', 'Football...",68,Sports,0,2018-04-24 17:16:49,Pro Football,"“I understand that they could meet with us, pa...",The New York Times,News,https://www.nytimes.com/2018/04/24/sports/foot...
1,5adf653f068401528a2aa697,656,By LISA FRIEDMAN,article,E.P.A. to Unveil a New Rule. Its Effect: Less ...,"['Environmental Protection Agency', 'Pruitt, S...",68,Climate,0,2018-04-24 17:11:21,Unknown,The agency plans to publish a new regulation T...,The New York Times,News,https://www.nytimes.com/2018/04/24/climate/epa...
2,5adf4626068401528a2aa628,2427,By PETE WELLS,article,"The New Noma, Explained","['Restaurants', 'Noma (Copenhagen, Restaurant)...",66,Dining,0,2018-04-24 14:58:44,Unknown,What’s it like to eat at the second incarnatio...,The New York Times,News,https://www.nytimes.com/2018/04/24/dining/noma...
3,5adf40d2068401528a2aa619,626,By JULIE HIRSCHFELD DAVIS and PETER BAKER,article,Unknown,"['Macron, Emmanuel (1977- )', 'Trump, Donald J...",68,Washington,0,2018-04-24 14:35:57,Europe,President Trump welcomed President Emmanuel Ma...,The New York Times,News,https://www.nytimes.com/2018/04/24/world/europ...
4,5adf3d64068401528a2aa60f,815,By IAN AUSTEN and DAN BILEFSKY,article,Unknown,"['Toronto, Ontario, Attack (April, 2018)', 'Mu...",68,Foreign,0,2018-04-24 14:21:21,Canada,"Alek Minassian, 25, a resident of Toronto’s Ri...",The New York Times,News,https://www.nytimes.com/2018/04/24/world/canad...


In [15]:
print('열의 개수: ', len(df.columns))

print(df.columns)

열의 개수:  15
Index(['articleID', 'articleWordCount', 'byline', 'documentType', 'headline',
       'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')


In [16]:
df['headline'].isnull().values.any()

False

In [17]:
headline = []
headline.extend(list(df.headline.values))
headline[:5]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown']

In [18]:
# headline 노이즈 제거
print('총 샘플 개수: {}'.format(len(headline)))

headline = [n for n in headline if n != "Unknown"]
print('노이즈값 제거 후 샘플 개수: {}'.format(len(headline)))

headline[:5]

총 샘플 개수: 1324
노이즈값 제거 후 샘플 개수: 1214


['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'How a Bag of Texas Dirt  Became a Times Tradition',
 'Is School a Place for Self-Expression?']

In [19]:
# 구두점 제거와 단어 소문자화
def repreprocessing(s):
    s = s.encode("utf8").decode("ascii",'ignore')
    return ''.join(c for c in s if c not in punctuation).lower()

text = [repreprocessing(x) for x in headline]
text[:5]

['former nfl cheerleaders settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'the new noma explained',
 'how a bag of texas dirt  became a times tradition',
 'is school a place for selfexpression']

In [20]:
# 단어 집합 생성
t = Tokenizer()
t.fit_on_texts(text)

vocab_size = len(t.word_index) + 1
print('단어 집합 크기: %d' % vocab_size)

단어 집합 크기: 3494


In [21]:
# 레이블을 분리하지 않은 훈련 데이터
sequences = list()
for line in text: 
    encoded = t.texts_to_sequences([line])[0] # 각 샘플에 대한 정수 인코딩
    
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)
        
sequences[:11] 

[[99, 269],
 [99, 269, 371],
 [99, 269, 371, 1115],
 [99, 269, 371, 1115, 582],
 [99, 269, 371, 1115, 582, 52],
 [99, 269, 371, 1115, 582, 52, 7],
 [99, 269, 371, 1115, 582, 52, 7, 2],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10, 1116],
 [100, 3]]

In [22]:
# 정수가 의미하는 단어 확인
index_to_word = {}
for key, value in t.word_index.items():
    index_to_word[value] = key
    
print('빈도수 상위 582번 단어: {}'.format(index_to_word[582]))

빈도수 상위 582번 단어: offer


In [23]:
# padding
max_len = max(len(l) for l in sequences)
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')

print(sequences[:3])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0   99  269]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0   99  269  371]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0   99  269  371 1115]]


In [24]:
# 샘플 마지막 단어를 레이블로 분리
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]

print(X[:3])
print(y[:3])

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0  99]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0  99 269]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0  99 269 371]]
[ 269  371 1115]


In [25]:
# 레이블 원-핫 인코딩
y = to_categorical(y, num_classes=vocab_size)

> **2) 모델 설계하기**

In [26]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM

In [27]:
model = Sequential()
# 단어 임베딩 벡터는 10차원, 은닉 상태 크기는 128인 LSTM
model.add(Embedding(vocab_size, 10, input_length=max_len-1))
model.add(LSTM(128))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=200, verbose=2)

Epoch 1/200
244/244 - 7s - loss: 7.6464 - accuracy: 0.0293
Epoch 2/200
244/244 - 5s - loss: 7.1158 - accuracy: 0.0302
Epoch 3/200
244/244 - 5s - loss: 6.9784 - accuracy: 0.0358
Epoch 4/200
244/244 - 5s - loss: 6.8471 - accuracy: 0.0424
Epoch 5/200
244/244 - 5s - loss: 6.6849 - accuracy: 0.0438
Epoch 6/200
244/244 - 5s - loss: 6.4958 - accuracy: 0.0478
Epoch 7/200
244/244 - 5s - loss: 6.2942 - accuracy: 0.0540
Epoch 8/200
244/244 - 5s - loss: 6.0860 - accuracy: 0.0559
Epoch 9/200
244/244 - 5s - loss: 5.8816 - accuracy: 0.0618
Epoch 10/200
244/244 - 5s - loss: 5.6898 - accuracy: 0.0677
Epoch 11/200
244/244 - 5s - loss: 5.5107 - accuracy: 0.0713
Epoch 12/200
244/244 - 5s - loss: 5.3420 - accuracy: 0.0788
Epoch 13/200
244/244 - 5s - loss: 5.1832 - accuracy: 0.0880
Epoch 14/200
244/244 - 5s - loss: 5.0343 - accuracy: 0.0953
Epoch 15/200
244/244 - 5s - loss: 4.8904 - accuracy: 0.1051
Epoch 16/200
244/244 - 5s - loss: 4.7529 - accuracy: 0.1174
Epoch 17/200
244/244 - 5s - loss: 4.6204 - accura

Epoch 137/200
244/244 - 12s - loss: 0.3476 - accuracy: 0.9144
Epoch 138/200
244/244 - 12s - loss: 0.3377 - accuracy: 0.9155
Epoch 139/200
244/244 - 12s - loss: 0.3358 - accuracy: 0.9152
Epoch 140/200
244/244 - 12s - loss: 0.3291 - accuracy: 0.9161
Epoch 141/200
244/244 - 12s - loss: 0.3270 - accuracy: 0.9152
Epoch 142/200
244/244 - 12s - loss: 0.3216 - accuracy: 0.9175
Epoch 143/200
244/244 - 12s - loss: 0.3205 - accuracy: 0.9164
Epoch 144/200
244/244 - 12s - loss: 0.3166 - accuracy: 0.9166
Epoch 145/200
244/244 - 12s - loss: 0.3231 - accuracy: 0.9152
Epoch 146/200
244/244 - 12s - loss: 0.3259 - accuracy: 0.9161
Epoch 147/200
244/244 - 12s - loss: 0.3217 - accuracy: 0.9152
Epoch 148/200
244/244 - 12s - loss: 0.3156 - accuracy: 0.9150
Epoch 149/200
244/244 - 12s - loss: 0.3079 - accuracy: 0.9168
Epoch 150/200
244/244 - 12s - loss: 0.3080 - accuracy: 0.9164
Epoch 151/200
244/244 - 12s - loss: 0.3027 - accuracy: 0.9167
Epoch 152/200
244/244 - 12s - loss: 0.2991 - accuracy: 0.9167
Epoch 15

In [28]:
def sentence_generation(model, t, current_word, n):
    init_word = current_word
    sentence = ''
    
    for _ in range(n):
        encoded = t.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen=23, padding='pre')
        result = model.predict_classes(encoded, verbose=0)
        
        for word, index in t.word_index.items():
            if index == result:
                break
        current_word = current_word + ' ' + word
        sentence = sentence + ' ' + word
        
    sentence = init_word + sentence
    return sentence

In [29]:
print(sentence_generation(model, t, 'i', 10))

C:\Users\USER\anaconda3\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


i want to be rich and im not sorry think beyond


In [30]:
print(sentence_generation(model, t, 'how', 10))

how to make a crossword puzzle kit its savory up of
